<h1 align="center">Lecture</h1>

In [39]:
# Большое спасибо Саше Панину, https://github.com/justheuristic

## Рекуррентные Нейронные Сети

<img src="pic/rnn.png" width="600">

## Backprop Through Time

<img src="pic/bptt.png" width="600">


## Truncated Backprop Through Time

<img src="pic/tbptt.png" width="600">


## Char RNN (Embeding Layer vs One Hot Encoding)

<img src="pic/crnn.png" width="600">


## Captioning (Embeding Layer vs One Hot Encoding)

<img src="pic/cap.png" width="600">

<h1 align="center">Char RNN</h1>

In [1]:
import os
import theano
import lasagne
import numpy as np
import theano.tensor as T

## Прочитаем кучу законов

In [5]:
#тут будет текст
corpora = ""

for fname in os.listdir("codex"):
    with open("codex/"+fname, encoding='cp1251') as fin:
        corpora += fin.read()

In [6]:
print('всего букв', len(corpora), 'тип', type(corpora))

всего букв 3331133 тип <class 'str'>


In [7]:
corpora[500:800]

'существление правосудия арбитражными судами\n\n Правосудие в сфере предпринимательской и иной экономической деятельности осуществляется арбитражными судами в Российской Федерации, образованными в соответствии с Конституцией Российской Федерации и федеральным конституционным законом (далее – арбитражны'

In [8]:
print(corpora[500:800])

существление правосудия арбитражными судами

 Правосудие в сфере предпринимательской и иной экономической деятельности осуществляется арбитражными судами в Российской Федерации, образованными в соответствии с Конституцией Российской Федерации и федеральным конституционным законом (далее – арбитражны


## Предобработаем текст

In [17]:
# тут будут все уникальные токены (буквы, цифры)
tokens = set(corpora)
tokens = list(tokens)

In [18]:
len(tokens)

101

In [19]:
# проверка на количество таких символов. Проверено на Python 2.7.11 Ubuntux64. 
assert(len(tokens) == 101)

In [24]:
# словарь символ -> его номер
token_to_id = {symbol: symbol_id for symbol_id, symbol in enumerate(tokens)}

# словарь номер символа -> сам символ
id_to_token = {symbol_id: symbol for symbol_id, symbol in enumerate(tokens)}

# Преобразуем всё в токены
corpora_ids = [token_to_id[symbol] for symbol in corpora]

In [29]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in range(n_batches):
        pos = np.random.randint(0, len(source) - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

In [30]:
X, y = sample_random_batches(corpora_ids, 4, 5)
print(X)
print(y)

[[ 66.  20.  51.  87.  70.]
 [ 66.   6.   0.  90.  74.]
 [  0.  38.  70.  90.  92.]
 [ 83.   0.  51.   6.  20.]]
[ 66.  56.  66.  38.]


## Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, и выдаёт вероятности для seq_len+1-ого.

Общий шаблон архитектуры такой сети:
* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.

Для обработки входных данных можно использовать либо EmbeddingLayer

In [50]:
seq_length = 5 # Как далеко распространяются градиенты
grad_clip = 10 # Максимальный модуль градиента
input_sequence, target_values = T.matrix('input sequence', 'int32'),  T.ivector('target y')

In [51]:
import lasagne.layers as ll

In [69]:
net = ll.InputLayer(shape=(None, None), input_var=input_sequence)

net = ll.EmbeddingLayer(net, len(tokens), 101)
net = ll.RecurrentLayer(net, 50, only_return_final=True, grad_clipping=grad_clip)

net = ll.DenseLayer(net, 101)

net = ll.DenseLayer(net, num_units=len(tokens), nonlinearity=lasagne.nonlinearities.softmax)

In [72]:
# Веса модели
weights = lasagne.layers.get_all_params(net, trainable=True)
print(weights)

[W, input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W, W, b, W, b]


In [75]:
network_output = ll.get_output(net)

In [76]:
loss = lasagne.objectives.categorical_crossentropy(network_output, target_values).mean()
updates = lasagne.updates.adam(loss, weights)

## Компилируем всякое-разное

In [77]:
# обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

# функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence], network_output, allow_input_downcast=True)

/usr/local/lib/python3.6/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


## Генерируем свои законы

 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью

In [80]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    
    return np.random.choice(np.arange(len(probs)), p=probs)

In [86]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.

def generate_sample(sample_fun, seed_phrase=None, N=200):
    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print("Using random seed:",seed_phrase)
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
#     assert(type(seed_phrase) is unicode)
           
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

## Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.

In [87]:
n_epochs=100 #сколько всего эпох
batches_per_epoch = 1000 # раз в сколько эпох печатать примеры 
batch_size=100 #сколько цепочек обрабатывать за 1 вызов функции обучения

print("Training ...")
for epoch in range(n_epochs):
    print("Генерируем текст в пропорциональном режиме")
    generate_sample(proportional_sample_fun,None)
    
    print("Генерируем текст в жадном режиме (наиболее вероятные буквы)")
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

Training ...
Генерируем текст в пропорциональном режиме
Using random seed: ом су


TypeError: Bad input argument to theano function with name "<ipython-input-77-fb24dc9036d8>:8" at index 0 (0-based).  
Backtrace when that variable is created:

  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 399, in execute_request
    user_expressions, allow_stdin)
  File "/usr/local/lib/python3.6/site-packages/ipykernel/ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/usr/local/lib/python3.6/site-packages/ipykernel/zmqshell.py", line 533, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2717, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/usr/local/lib/python3.6/site-packages/IPython/core/int

## Конституция нового мирового правительства

In [ ]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

In [ ]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

# не забывайте оставлять отзывы 
# о лекции https://goo.gl/gMeYNL о семинаре https://goo.gl/5hlPD0 :)